In [2]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

In [3]:
rating_data = pd.read_csv('ratings.csv',parse_dates=['timestamp'])
movies_data = pd.read_csv('movies.csv')

In [4]:
rating_data.info()
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   userId     1048575 non-null  int64  
 1   movieId    1048575 non-null  int64  
 2   rating     1048575 non-null  float64
 3   timestamp  1048575 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 32.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [5]:
user_ID = rating_data["userId"].unique().tolist()
total_users = len(user_ID)
np.random.seed(236)
random_user_id = np.random.choice(user_ID, size = int(total_users * 0.1), replace=False)
user_data = rating_data.loc[rating_data['userId'].isin(random_user_id)]
user_data.head()

,userId,movieId,rating,timestamp
2862,14,50,5.0,1506209168
2863,14,293,5.0,1506209067
2864,14,296,5.0,1506208897
2865,14,527,5.0,1506208872
2866,14,608,5.0,1506209155


In [6]:
def movie_matrix(data):
    matrix = data.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
    matrix_comp = csr_matrix(matrix.values)
    return matrix, matrix_comp
rating_matrix, comp_rating_matrix = movie_matrix(user_data)

In [7]:
def latent_feature(dim, matrix_cp):
    mf_nn = NMF(n_components=dim)
    mf_nn.fit(matrix_cp)
    vec_item = mf_nn.components_.T
    return vec_item

In [2]:
import surprise
from surprise.model_selection import cross_validate
from surprise import NMF as surprise_nmf
from surprise.accuracy import rmse
from surprise import accuracy
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split as surprise_train_test_split

ModuleNotFoundError: No module named 'surprise'

In [ ]:
def rmse_mf_nn(sample_data, num_factors):
    client = Reader(rating_scale=(0, 5))
    dt = Dataset.load_from_df(sample_data[['userId', 'movieId', 'rating']], client)
    train_data, test_data = surprise_train_test_split(dt, test_size=0.3)
    algo = surprise_nmf(n_factors = num_factors)
    predictions = algo.fit(train_data).test(test_data)
    accuracy.rmse(predictions)

In [ ]:
rmse_mf_nn(user_data, 10)

NameError: name 'Reader' is not defined